<a href="https://colab.research.google.com/github/Pushkal-B/WEATHER-REPO/blob/main/Wether.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
from datetime import datetime
from tabulate import tabulate
import time
import sys
import threading
import json
import os
from functools import wraps

In [32]:


# Enhanced city mappings with more information
CITY_MAPPING = {
    'NYC': {'name': 'New York', 'country': 'US', 'timezone': 'America/New_York'},
    'LA': {'name': 'Los Angeles', 'country': 'US', 'timezone': 'America/Los_Angeles'},
    'SF': {'name': 'San Francisco', 'country': 'US', 'timezone': 'America/Los_Angeles'},
    'CHI': {'name': 'Chicago', 'country': 'US', 'timezone': 'America/Chicago'},
    'UK': {'name': 'London', 'country': 'GB', 'timezone': 'Europe/London'},
    'JP': {'name': 'Tokyo', 'country': 'JP', 'timezone': 'Asia/Tokyo'},
    'IND': {'name': 'New Delhi', 'country': 'IN', 'timezone': 'Asia/Kolkata'},
    'UAE': {'name': 'Dubai', 'country': 'AE', 'timezone': 'Asia/Dubai'},
    'AUS': {'name': 'Sydney', 'country': 'AU', 'timezone': 'Australia/Sydney'},
    'CHN': {'name': 'Beijing', 'country': 'CN', 'timezone': 'Asia/Shanghai'},
    'RUS': {'name': 'Moscow', 'country': 'RU', 'timezone': 'Europe/Moscow'},
    'BRA': {'name': 'Sao Paulo', 'country': 'BR', 'timezone': 'America/Sao_Paulo'},
    'CAN': {'name': 'Toronto', 'country': 'CA', 'timezone': 'America/Toronto'}
}

def with_loading_animation(func):
    """Enhanced loading animation decorator"""
    @wraps(func)
    def wrapper(*args, **kwargs):
        stop_event = threading.Event()

        def spinner_task():
            spinner = ['|', '/', '-', '\\', '|']
            while not stop_event.is_set():
                for char in spinner:
                    if stop_event.is_set():
                        break
                    sys.stdout.write('\r' + char)
                    sys.stdout.flush()
                    time.sleep(0.1)
            sys.stdout.write('\r')
            sys.stdout.flush()

        spinner_thread = threading.Thread(target=spinner_task)
        spinner_thread.daemon = True
        spinner_thread.start()

        start_time = time.time()

        try:
            result = func(*args, **kwargs)
            execution_time = time.time() - start_time
            if execution_time < 3:
                time.sleep(3 - execution_time)
            return result
        finally:
            stop_event.set()
            spinner_thread.join()

    return wrapper

def save_to_history(weather_data, filename="weather_history.json"):
    """Save weather data to history file"""
    try:
        if os.path.exists(filename):
            with open(filename, 'r') as f:
                history = json.load(f)
        else:
            history = []

        current_data = {
            'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'data': weather_data
        }
        history.append(current_data)

        # Keep only last 10 queries
        if len(history) > 10:
            history = history[-10:]

        with open(filename, 'w') as f:
            json.dump(history, f, indent=2)
    except Exception as e:
        print(f"Error saving history: {e}")

@with_loading_animation
def get_weather_data(cities, api_key):
    """Fetch weather data for multiple cities"""
    weather_data = []
    errors = []

    for city in cities:
        try:
            # Get full city name and info
            city_info = CITY_MAPPING.get(city.upper(), {'name': city, 'country': None})
            city_name = city_info['name']

            complete_api_link = f"https://api.openweathermap.org/data/2.5/weather?q={city_name}&appid={api_key}"
            api_link = requests.get(complete_api_link)
            api_data = api_link.json()

            if api_link.status_code != 200:
                errors.append(f"Error fetching data for {city} ({city_name}): {api_data.get('message', 'Unknown error')}")
                continue

            # Enhanced weather data collection
            weather_info = {
                'city': city_name,
                'temperature': round(api_data['main']['temp'] - 273.15, 2),
                'description': api_data['weather'][0]['description'],
                'humidity': api_data['main']['humidity'],
                'wind_speed': round(api_data['wind']['speed'] * 3.6, 1),
                'pressure': api_data['main'].get('pressure', 'N/A'),
                'feels_like': round(api_data['main'].get('feels_like', 0) - 273.15, 2),
                'visibility': api_data.get('visibility', 'N/A'),
                'sunrise': datetime.fromtimestamp(api_data['sys']['sunrise']).strftime('%H:%M'),
                'sunset': datetime.fromtimestamp(api_data['sys']['sunset']).strftime('%H:%M'),
                'country': api_data['sys']['country']
            }
            weather_data.append(weather_info)

        except Exception as e:
            errors.append(f"Error processing {city}: {str(e)}")

    return weather_data, errors

def create_fancy_table(data, headers):
    """Create an enhanced table with proper alignment and formatting"""
    alignments = {
        'City': 'left',
        'Temp (°C)': 'right',
        'Feels Like (°C)': 'right',
        'Description': 'left',
        'Humidity (%)': 'right',
        'Wind (km/h)': 'right',
        'Pressure (hPa)': 'right',
        'Visibility (m)': 'right',
        'Sunrise': 'center',
        'Sunset': 'center'
    }
    colalign = [alignments[header] for header in headers]
    return tabulate(data, headers=headers, tablefmt='fancy_grid', colalign=colalign, floatfmt='.1f')

def parse_input(input_string):
    """Parse input string containing cities separated by comma or semicolon"""
    cities = input_string.replace(';', ',').split(',')
    return [city.strip() for city in cities if city.strip()]

def show_history(filename="weather_history.json"):
    """Display weather query history"""
    try:
        if not os.path.exists(filename):
            print("No history available yet.")
            return

        with open(filename, 'r') as f:
            history = json.load(f)

        print("\n╔══════════════════════╗")
        print("║   Weather History    ║")
        print("╚══════════════════════╝")

        for entry in history:
            print(f"\nQuery Time: {entry['timestamp']}")
            cities = [data['city'] for data in entry['data']]
            print(f"Cities queried: {', '.join(cities)}")

    except Exception as e:
        print(f"Error reading history: {e}")

def main():
    api_key = '87d845b0b6cf29baa1a73cc34b067a95'

    while True:
        print("\n╔═════════════════════════════════════════════════════════╗")
        print("║             Weather Information System                  ║")
        print("╚═════════════════════════════════════════════════════════╝")

        print("\n1. Check Weather")
        print("2. View History")
        print("3. Exit")

        choice = input("\nEnter your choice (1-3): ")

        if choice == '1':
            print("\nEnter cities or common abbreviations (separated by commas or semicolons)")
            print("Supported abbreviations:", ", ".join(CITY_MAPPING.keys()))
            print("Examples: 'NYC, UK, JP' or 'London;Paris;Tokyo'")

            cities_input = input("\nEnter locations: ")
            cities = parse_input(cities_input)

            if not cities:
                print("No valid locations entered.")
                continue

            weather_data, errors = get_weather_data(cities, api_key)

            if weather_data:
                headers = ['City', 'Temp (°C)', 'Feels Like (°C)', 'Description',
                          'Humidity (%)', 'Wind (km/h)', 'Pressure (hPa)',
                          'Visibility (m)', 'Sunrise', 'Sunset']

                table_data = [[
                    data['city'],
                    data['temperature'],
                    data['feels_like'],
                    data['description'].title(),
                    data['humidity'],
                    data['wind_speed'],
                    data['pressure'],
                    data['visibility'],
                    data['sunrise'],
                    data['sunset']
                ] for data in weather_data]

                current_time = datetime.now().strftime('%d %b %Y | %I:%M:%S %p')
                print(f"\nWeather Report - {current_time}\n")
                print(create_fancy_table(table_data, headers))

                # Save to history
                save_to_history(weather_data)

                # Save current report
                with open("weather_report.txt", "w", encoding='utf-8') as f:
                    f.write(f"Weather Report - Generated on {current_time}\n\n")
                    f.write(create_fancy_table(table_data, headers))
                    f.write("\n\nEnd of Report")

                print(f"\nReport saved to 'weather_report.txt'")

            if errors:
                print("\n╔════════════════════╗")
                print("║ Errors Encountered ║")
                print("╚════════════════════╝")
                for error in errors:
                    print(f"• {error}")

        elif choice == '2':
            show_history()

        elif choice == '3':
            print("\nThank you for using the Weather Search!\n by- PK")
            break

        else:
            print("\nInvalid choice. Please try again.")

if __name__ == "__main__":
    main()

/
╔═════════════════════════════════════════════════════════╗
║             Weather Information System                  ║
╚═════════════════════════════════════════════════════════╝

1. Check Weather
2. View History
3. Exit
\
Enter your choice (1-3): 1
|
Enter cities or common abbreviations (separated by commas or semicolons)
Supported abbreviations: NYC, LA, SF, CHI, UK, JP, IND, UAE, AUS, CHN, RUS, BRA, CAN
Examples: 'NYC, UK, JP' or 'London;Paris;Tokyo'
\
Enter locations: nyc

Weather Report - 11 Nov 2024 | 01:59:09 AM

╒══════════╤═════════════╤═══════════════════╤═════════════════╤════════════════╤═══════════════╤══════════════════╤══════════════════╤═══════════╤══════════╕
│ City     │   Temp (°C) │   Feels Like (°C) │ Description     │   Humidity (%) │   Wind (km/h) │   Pressure (hPa) │   Visibility (m) │  Sunrise  │  Sunset  │
╞══════════╪═════════════╪═══════════════════╪═════════════════╪════════════════╪═══════════════╪══════════════════╪══════════════════╪═══════════╪══════